In [6]:
! pip install -q langchainhub

### Imports

In [59]:
from langchain import hub
from langchain_community.chat_models import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document



from operator import itemgetter


from getpass import getpass


### Set the prompt

In [9]:
prompt = hub.pull("langchain-ai/retrieval-qa-chat")
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

### Set the llm

In [107]:
OPENAI_API_KEY = getpass()

In [12]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [15]:
llm = ChatOpenAI()

### Set the embeddings

In [23]:
embeddings = HuggingFaceEmbeddings(model_kwargs={'device' : 'cpu'})

### Set the vector index

In [108]:
PINECONE_API_KEY = getpass()

In [17]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [25]:
index_name = "tamil-movies-4k"
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
retriever = docsearch.as_retriever()

### Memory

In [34]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

### Set the question prompt

In [43]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [100]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [47]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

### Set the Answer prompt

In [53]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

### Create conversation chain

In [101]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

In [102]:
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}

In [103]:
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

In [104]:
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

In [105]:
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
#     "docs": itemgetter("docs"),
}

In [106]:
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer


### Inference

In [94]:
inputs = {"question": "List the last 3 Vijay movie names"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Thuppakki (2012), Kaththi (2014) and Puli (2015)', response_metadata={'finish_reason': 'stop', 'logprobs': None})}

In [99]:
result["answer"].content

'Thuppakki (2012), Kaththi (2014) and Puli (2015)'

In [80]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [81]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='List the last 3 Vijay movie names'), AIMessage(content='The names of the last 3 movies released by Vijay are Thuppakki (2012), Kaththi (2014), and Puli (2015).')]), output_key='answer', input_key='question', return_messages=True)

In [82]:
# inputs = {"question": "Who directed the second movie"}
result = final_chain.invoke({

    "question": "who directed the second movie?",
    "chat_history": memory.load_memory_variables({})["history"],
    
})
result

{'answer': AIMessage(content='AR Murugadoss', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'docs': [Document(page_content="Murugadoss replaced his usual music director Harris Jayaraj with Anirudh Ravichander signing for the project. George C. Williams, who worked on the director's production Raja Rani (2013), was signed as the cinematographer. Art director Lalgudi N. Ilayaraja was selected to handle the production design. The film was earlier rumoured to be titled as Dheeran and Vaal. However, in March 2014, Murugadoss confirmed that the film would be titled Kaththi and that it was scheduled to be released on Diwali. In June", metadata={'id': '41839006', 'release_year': '2014', 'row': 2241.0, 'source': 'movies_update.csv', 'title': 'Kaththi', 'url': 'https://en.wikipedia.org/wiki/Kaththi'}),
  Document(page_content="Made on a budget of 70 crore, Kaththi was released worldwide on 22 October 2014, coinciding with Diwali, and received critical acclaim with critics prai

In [83]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [84]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='List the last 3 Vijay movie names'), AIMessage(content='The names of the last 3 movies released by Vijay are Thuppakki (2012), Kaththi (2014), and Puli (2015).'), HumanMessage(content='List the last 3 Vijay movie names'), AIMessage(content='AR Murugadoss')]), output_key='answer', input_key='question', return_messages=True)

In [95]:
memory.clear()

In [96]:
async for chunk in final_chain.astream({"question": "List the last 3 Vijay movies"}):
    print(chunk, end="|", flush=True)

{'answer': AIMessageChunk(content='')}|{'answer': AIMessageChunk(content='Th')}|{'answer': AIMessageChunk(content='upp')}|{'answer': AIMessageChunk(content='ak')}|{'answer': AIMessageChunk(content='ki')}|{'answer': AIMessageChunk(content=' (')}|{'answer': AIMessageChunk(content='201')}|{'answer': AIMessageChunk(content='2')}|{'answer': AIMessageChunk(content='),')}|{'answer': AIMessageChunk(content=' Kath')}|{'answer': AIMessageChunk(content='th')}|{'answer': AIMessageChunk(content='i')}|{'answer': AIMessageChunk(content=' (')}|{'answer': AIMessageChunk(content='201')}|{'answer': AIMessageChunk(content='4')}|{'answer': AIMessageChunk(content=')')}|{'answer': AIMessageChunk(content=' and')}|{'answer': AIMessageChunk(content=' P')}|{'answer': AIMessageChunk(content='uli')}|{'answer': AIMessageChunk(content=' (')}|{'answer': AIMessageChunk(content='201')}|{'answer': AIMessageChunk(content='5')}|{'answer': AIMessageChunk(content=')')}|{'answer': AIMessageChunk(content='', response_metadata